# **Generating CBR DataFrame**

In [1]:
import sys
import math
import pandas as pd
from natsort import natsorted
import os
from tqdm import tqdm
pd.set_option('display.max_rows', None)

##### **Setting File Names**

In [2]:
directory = 'Highway3km'
ipd_file_name = '0ipdqtest.txt'
ids_file_name = '0idsqtest.txt'

##### **Transmission Powers**

In [3]:
arr = natsorted(os.listdir(directory))
arr

['2mW', '4mW', '6mW', '8mW', '10mW', '12mW', '14mW', '16mW', '18mW', '20mW']

##### **Next Line Function**

In [4]:
def next_line(file):
    line = file.readline()
    return line

##### **Lists for Storing Data**

In [11]:
trans_power = []
vehicle_density = []
ipd_list = []
range_km = []

##### **Generating IDP Values and Appending Data to Lists**

In [12]:
for filename_1 in tqdm(natsorted(os.listdir(directory))):
    print(filename_1)
    f = os.path.join(directory, filename_1)
    for filename_2 in tqdm(os.listdir(f)):
        print(filename_2)
        b = os.path.join(f, filename_2) 
        g = os.path.join(b, ipd_file_name)
        c = os.path.join(b, ids_file_name)

        # file containing distance and position stats
        file = open(g, "r")
        data = next_line(file).strip('\n')

        numVehs = 0

        # file containing the vehicle IDs where each line is in the form x_y, where
        # x = mobility->getExternalId() and the module name in the .anf file is
        # ConfigCollectionnetwork.node[y].appl
        ids = open(c, "r")
        vDict = {} # this will contain the link between x and y in each line in the ID file

        idLine = next_line(ids).strip('\n')
        while idLine:
            u = idLine.find('_')
            vId = idLine[u+1:]
            fId = idLine[0:u]
            vDict[fId] = eval(vId)
            idLine = next_line(ids).strip('\n')
            numVehs += 1

        ipdList = [] # contains IPD data for each sender-receiver vehicle pair
        for i in range(0, numVehs):
            ipdList.append([-1 for j in range(0, numVehs)])
        #print("Now scanning input")

        # this file can be used to write each IPD and distance value to; can be commented out if not needed
        #output = open("output4lanetenhz20mw200.txt", "w")

        # distance ranges; these are optional and can be commented out
        # to be used when you want the total number of BSMs received within 50m,
        # 100m, 200m, 300m, and >300m by all vehicles
        n0 = 0
        n50 = 0
        n100 = 0
        n200 = 0
        n300 = 0

        # this list is to be used if you want to get the average IPD for consistent
        # distance intervals; number of intervals can be changed as needed.
        ipdIntervals = [[] for i in range(0,35)]

        # individual vehicle IPD data; these are optional and can be commented out
        # to be used when you want to get IPD data on specific vehicles
        ipd50 = []
        ipd100 = []
        ipd150 = []
        ipd200 = []
        ipd539 = []

        maxDist = 0
        maxGap = 0
        last = -1

        j = 1
        while data:
            if data[:10] == 'undisposed':
                data = next_line(file).strip('\n')
                continue

            # divides each line in the distance file into the individual data pieces
            first = data.find('_')
            second = data.find('_', first+1)
            third = data.find('_', second+1)
            fourth = data.find('_', third+1)
            fifth = data.find('_', fourth+1)
            sixth = data.find('_', fifth+1)
            seventh = data.find('_', sixth+1)
            senderX = eval(data[:first])
            senderY = eval(data[first+1:second])
            senderID = data[second+1:third]
            receiverX = eval(data[third+1:fourth])
            receiverY = eval(data[fourth+1:fifth])
            receiverID = data[fifth+1:sixth]
            simTime = eval(data[seventh+1:])
            distance = round(math.sqrt((senderX-receiverX)**2 + (senderY-receiverY)**2), 5) # calculates distance between sending and receiving vehicle

            """if last == -1:
                last = simTime
            else:
                if simTime - last > maxGap:
                    maxGap = simTime - last
                last = simTime"""

        # this block is only to be used with the distance range variables; otherwise it can be commented out
        # the distance values can be changed as needed
            if distance <= 50:
                n0 += 1
            elif distance <= 100:
                n50 += 1
            elif distance <= 200:
                n100 += 1
            elif distance <= 300:
                n200 += 1
            else:
                n300 += 1

            lastTime = ipdList[vDict[receiverID]][vDict[senderID]]
            if lastTime != -1:
                ipd = simTime - lastTime
            else:
                ipd = -1

            if distance > maxDist:
                maxDist = distance

        # this section is only to be used with the individual vehicle IPD data; otherwise it can be commented out
        # vehicle names and if condition can be changed as needed
                """if senderID == 'sender' and 'receiver' in receiverID:
                    if receiverID == 'receiver50':
                        ipd50.append(ipd)
                    elif receiverID == 'receiver100':
                        ipd100.append(ipd)
                    elif receiverID == 'receiver150':
                        ipd150.append(ipd)
                    elif receiverID == 'receiver200':
                        ipd200.append(ipd)
                    elif receiverID == 'receiver539':
                        ipd539.append(ipd)"""

        # this section is only used if you are writing the IPD and distance values
        # to the output file; otherwise it can be commented out
                """output.write(senderID + '\t' + receiverID + '\t' + str(distance) + '\t' + str(ipd) + '\n')
                output.flush()"""

        # this section is used only if you want to get the average IPD over consistent
        # distance intervals; otherwise it can be commented out
        # the number of intervals and length of each interval can be changed as needed
            for i in range(1, len(ipdIntervals)+1):
                if 20*i > distance and ipd != -1:
                    ipdIntervals[i-1].append(ipd)
                    break

        # this section is only used if you are writing the IPD and distance values
        # to the output file and only if you want to include the first time a
        # sender-receiver pair exchanges BSMs; otherwise it can be commented out		
            """else:
                output.write(senderID + '\t' + receiverID + '\t' + str(distance) + '\t' + "First" + '\n')"""


            ipdList[vDict[receiverID]][vDict[senderID]] = simTime
            data = next_line(file).strip('\n')

            # this block tells you how many lines the program has gone through so you can
            # see how much progress the program is making
            j+=1
            #if j%100000 == 0:
                #print(j)

        # print IPD data for individual vehicles if used
        """print(sum(ipd50)/len(ipd50), len(ipd50))
        print(sum(ipd100)/len(ipd100), len(ipd100))
        print(sum(ipd150)/len(ipd150), len(ipd150))
        print(sum(ipd200)/len(ipd200), len(ipd200))
        print(sum(ipd539)/len(ipd539), len(ipd539))"""

        # print number of BSMs received in distance ranges if used
        #print(n0, n50, n100, n200, n300)

        # print average IPD for each distance interval if ipdIntervals is used
        #print("**********")
        for i in range(0, len(ipdIntervals)):
            if len(ipdIntervals[i]) > 0:
                ipdIntervals[i] = sum(ipdIntervals[i])/len(ipdIntervals[i])
            else:
                ipdIntervals[i] = 'N/A'
            #print(str(i*20) + " - " + str((i+1)*20) + '\t' + str(ipdIntervals[i]))

        #print(maxGap)
        #print(maxDist)

        #file.close()
        #output.close()
        #ids.close()
        
        #ahsu code
        for i in range(0, len(ipdIntervals)):
            if(ipdIntervals[i] != 'N/A'):
                ipd_list.append(ipdIntervals[i])
                range_km.append(str(i*20) + " - " + str((i+1)*20))
                trans_power.append(filename_1)
                vehicle_density.append(filename_2)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

2mW



  0%|                                                                                           | 0/13 [00:00<?, ?it/s]

3km1



  8%|██████▍                                                                            | 1/13 [00:00<00:06,  1.88it/s]

...


 85%|█████████████████████████████████████████████████████████████████████▍            | 11/13 [04:40<01:11, 35.78s/it]

3km5



 92%|███████████████████████████████████████████████████████████████████████████▋      | 12/13 [04:49<00:27, 27.56s/it]

3km51



100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [49:42<00:00, 298.25s/it]


#### **Creating DataFrame**


In [13]:
df = pd.DataFrame(list(zip(trans_power, vehicle_density, ipd_list, range_km)),
               columns =['Transmission_Power', 'Vehicle_Density', 'IPD', 'Range_KM'], index=None)
df

,Transmission_Power,Vehicle_Density,IPD,Range_KM
0,2mW,3km1,0.100417,0 - 20
1,2mW,3km1,0.100998,120 - 140
2,2mW,3km11,0.109082,0 - 20
3,2mW,3km11,0.111092,60 - 80
4,2mW,3km11,0.127103,140 - 160
5,2mW,3km13,0.120468,0 - 20
6,2mW,3km13,0.122502,40 - 60
7,2mW,3km13,0.131643,100 - 120
8,2mW,3km13,0.167274,140 - 160
9,2mW,3km17,0.128405,0 - 20


In [14]:
df.shape

(1805, 4)

In [ ]:
file = open('HIghway3km\\2mW\\3km11\\0ipdqtest.txt', "r")
data = next_line(file).strip('\n')

numVehs = 0

# file containing the vehicle IDs where each line is in the form x_y, where
# x = mobility->getExternalId() and the module name in the .anf file is
# ConfigCollectionnetwork.node[y].appl
ids = open("HIghway3km\\2mW\\3km11\\0idsqtest.txt", "r")
vDict = {} # this will contain the link between x and y in each line in the ID file

idLine = next_line(ids).strip('\n')
while idLine:
    u = idLine.find('_')
    vId = idLine[u+1:]
    fId = idLine[0:u]
    vDict[fId] = eval(vId)
    idLine = next_line(ids).strip('\n')
    numVehs += 1

ipdList = [] # contains IPD data for each sender-receiver vehicle pair
for i in range(0, numVehs):
    ipdList.append([-1 for j in range(0, numVehs)])
print("Now scanning input")

# this file can be used to write each IPD and distance value to; can be commented out if not needed
#output = open("output4lanetenhz20mw200.txt", "w")

# distance ranges; these are optional and can be commented out
# to be used when you want the total number of BSMs received within 50m,
# 100m, 200m, 300m, and >300m by all vehicles
n0 = 0
n50 = 0
n100 = 0
n200 = 0
n300 = 0

# this list is to be used if you want to get the average IPD for consistent
# distance intervals; number of intervals can be changed as needed.
ipdIntervals = [[] for i in range(0,35)]

# individual vehicle IPD data; these are optional and can be commented out
# to be used when you want to get IPD data on specific vehicles
ipd50 = []
ipd100 = []
ipd150 = []
ipd200 = []
ipd539 = []

maxDist = 0
maxGap = 0
last = -1

j = 1
while data:
    if data[:10] == 'undisposed':
        data = next_line(file).strip('\n')
        continue

    # divides each line in the distance file into the individual data pieces
    first = data.find('_')
    second = data.find('_', first+1)
    third = data.find('_', second+1)
    fourth = data.find('_', third+1)
    fifth = data.find('_', fourth+1)
    sixth = data.find('_', fifth+1)
    seventh = data.find('_', sixth+1)
    senderX = eval(data[:first])
    senderY = eval(data[first+1:second])
    senderID = data[second+1:third]
    receiverX = eval(data[third+1:fourth])
    receiverY = eval(data[fourth+1:fifth])
    receiverID = data[fifth+1:sixth]
    simTime = eval(data[seventh+1:])
    distance = round(math.sqrt((senderX-receiverX)**2 + (senderY-receiverY)**2), 5) # calculates distance between sending and receiving vehicle

    """if last == -1:
        last = simTime
    else:
        if simTime - last > maxGap:
            maxGap = simTime - last
        last = simTime"""

# this block is only to be used with the distance range variables; otherwise it can be commented out
# the distance values can be changed as needed
    if distance <= 50:
        n0 += 1
    elif distance <= 100:
        n50 += 1
    elif distance <= 200:
        n100 += 1
    elif distance <= 300:
        n200 += 1
    else:
        n300 += 1

    lastTime = ipdList[vDict[receiverID]][vDict[senderID]]
    if lastTime != -1:
        ipd = simTime - lastTime
    else:
        ipd = -1

    if distance > maxDist:
        maxDist = distance

# this section is only to be used with the individual vehicle IPD data; otherwise it can be commented out
# vehicle names and if condition can be changed as needed
        """if senderID == 'sender' and 'receiver' in receiverID:
            if receiverID == 'receiver50':
                ipd50.append(ipd)
            elif receiverID == 'receiver100':
                ipd100.append(ipd)
            elif receiverID == 'receiver150':
                ipd150.append(ipd)
            elif receiverID == 'receiver200':
                ipd200.append(ipd)
            elif receiverID == 'receiver539':
                ipd539.append(ipd)"""

# this section is only used if you are writing the IPD and distance values
# to the output file; otherwise it can be commented out
        """output.write(senderID + '\t' + receiverID + '\t' + str(distance) + '\t' + str(ipd) + '\n')
        output.flush()"""

# this section is used only if you want to get the average IPD over consistent
# distance intervals; otherwise it can be commented out
# the number of intervals and length of each interval can be changed as needed
    for i in range(1, len(ipdIntervals)+1):
        if 20*i > distance and ipd != -1:
            ipdIntervals[i-1].append(ipd)
            break

# this section is only used if you are writing the IPD and distance values
# to the output file and only if you want to include the first time a
# sender-receiver pair exchanges BSMs; otherwise it can be commented out		
    """else:
        output.write(senderID + '\t' + receiverID + '\t' + str(distance) + '\t' + "First" + '\n')"""


    ipdList[vDict[receiverID]][vDict[senderID]] = simTime
    data = next_line(file).strip('\n')

    # this block tells you how many lines the program has gone through so you can
    # see how much progress the program is making
    j+=1
    if j%100000 == 0:
        print(j)

# print IPD data for individual vehicles if used
"""print(sum(ipd50)/len(ipd50), len(ipd50))
print(sum(ipd100)/len(ipd100), len(ipd100))
print(sum(ipd150)/len(ipd150), len(ipd150))
print(sum(ipd200)/len(ipd200), len(ipd200))
print(sum(ipd539)/len(ipd539), len(ipd539))"""

# print number of BSMs received in distance ranges if used
print(n0, n50, n100, n200, n300)

# print average IPD for each distance interval if ipdIntervals is used
print("**********")
for i in range(0, len(ipdIntervals)):
    if len(ipdIntervals[i]) > 0:
        ipdIntervals[i] = sum(ipdIntervals[i])/len(ipdIntervals[i])
    else:
        ipdIntervals[i] = 'N/A'
    print(str(i*20) + " - " + str((i+1)*20) + '\t' + str(ipdIntervals[i]))

#print(maxGap)
#print(maxDist)

file.close()
#output.close()
#ids.close()

In [21]:
for i in range(0, len(ipdIntervals)):
    if(ipdIntervals[i] != 'N/A'):
        print(i)
        print(ipdIntervals[i])
        print(str(i*20) + " - " + str((i+1)*20))
#ipdIntervals

0
0.19050428795307034
0 - 20
1
0.21003205150513798
20 - 40
2
0.2359348201513226
40 - 60
3
0.28512604088542115
60 - 80
4
0.35454122837583973
80 - 100
5
0.40868938102410407
100 - 120
6
0.5020100761337246
120 - 140
7
0.6134902517905422
140 - 160
8
0.6932910234546246
160 - 180
9
0.8148376137035797
180 - 200
10
0.945464239069686
200 - 220
11
1.0296261419385035
220 - 240
12
1.1534157060204102
240 - 260
13
1.2861892741687062
260 - 280
14
1.376345113121551
280 - 300
15
1.5228783032569524
300 - 320
16
1.6791467509812872
320 - 340
17
1.8144909598764833
340 - 360
18
1.9410472940821903
360 - 380
19
2.0419567863397723
380 - 400
20
2.138944434008101
400 - 420
21
2.218544318854171
420 - 440
22
2.2923951209309563
440 - 460
23
2.2237510245798204
460 - 480
24
2.2327677090425246
480 - 500
25
2.2036663167809176
500 - 520
26
1.5771921420268968
520 - 540
27
2.060673089102688
540 - 560
28
3.1626801243281255
560 - 580
29
3.2669619958809997
580 - 600
30
3.2018624173609997
600 - 620
